In [155]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
complete_data = study_results.merge(mouse_metadata, on="Mouse ID")

filtered_data = complete_data.drop_duplicates(subset="Mouse ID", keep= 'first', ignore_index= True)


# Display the data table for preview
filtered_data.head()


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [156]:
# Checking the number of mice.
filtered_data["Mouse ID"].count()

249

In [51]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

249

In [157]:
# Optional: Get all the data for the duplicate mouse ID. 
g989_data = complete_data[complete_data["Mouse ID"] == 'g989']	
g989_data

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
filt_mouse_data = filtered_data[filtered_data["Mouse ID"] != 'g989']
filt_mouse_data.head()

In [158]:
# Checking the number of mice in the clean DataFrame.
filt_mouse_data["Mouse ID"].count()

248

In [165]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.


mean_tumor =  complete_data.groupby(["Drug Regimen"])['Tumor Volume (mm3)'].mean()
median_tumor =  complete_data.groupby(["Drug Regimen"])['Tumor Volume (mm3)'].median()
var_tumor =  complete_data.groupby(["Drug Regimen"])['Tumor Volume (mm3)'].var()
std_tumor =  complete_data.groupby(["Drug Regimen"])['Tumor Volume (mm3)'].std()
sem_tumor =  complete_data.groupby(["Drug Regimen"])['Tumor Volume (mm3)'].sem()

mean2= pd.DataFrame(mean_tumor)
mean2.head()


TypeError: Index(...) must be called with a collection of some kind, 'Mean Tumor Volume' was passed